# Image Cropping and Augmentation Pipeline

This notebook demonstrates how to automatically crop random patches from a set of BMP images using Python.

We will use the following libraries:
- `Pillow` for image loading and saving
- `NumPy` for array manipulation
- `glob` and `os` for file management

**Chapters:**
1. Imports
2. Random Crop Function
3. Image Loader and Cropper
4. Running the Pipeline


## 1. Imports

Import required libraries for image processing and file management.

In [23]:
import os
from glob import glob
import numpy as np
from PIL import Image

In [24]:

# OpenCV Read
import cv2 as cv

from PIL import Image

img_cv = cv.imread("./img1.jpg")

if img_cv is None:
    raise "Error : Imahe not found"
    

print("shape (Height, width, channel): ",img_cv.shape)
print("image type: ", img_cv.dtype)
print("image size: ", img_cv.size)


try:    

    img_pil = Image.open("./img1.jpg")
    print(f"Pil image {img_pil.mode} , Size: {img_pil.size}")
except FileExistsError:
    print("Error: image not found")
# Pillow Read
    

shape (Height, width, channel):  (379, 512, 3)
image type:  uint8
image size:  582144
Pil image RGB , Size: (512, 379)


In [30]:
cropped = img_cv[0:300,0:500]
cv.imshow("Cropped ",cropped)
cv.waitKey(0)
cv.destroyAllWindows()


In [29]:
box = (0,0,500,300)
cropped_pil = img_pil.crop(box)
cropped_pil.show()

## Advance

## 2. Random Crop Function

This function takes a NumPy image array and returns a random crop of the given size. If the image is too small, it returns `None`.

In [31]:
def get_random_crop(image, crop_height=200, crop_width=200):
    
    img_h, img_w = image.shape[:2]
    
    if img_h <crop_height or img_w <crop_width:
        return None, False
    
    max_x = img_w - crop_width
    max_y = img_h - crop_height
    
    x = np.random.randint(0,max_x)
    y = np.random.randint(0,max_y)
    
    crop = image[y:y+crop_height, x:x+crop_width]
    
    return crop, True

    
    
    
    

## 3. Image Loader and Cropper

This function loads all BMP images from the given directory, takes random crops, and saves them as PNG files in a new 'Crop' subdirectory.

In [ ]:
def loader(img_dir, crop_size = 200, channles = 3, n_crop = 5):
    
    img_files = glob(os.path.join(img_dir,"*.png"))
    print(img_files)
    crop_dir = os.path.join(img_dir,"Cropped_Images")
    os.makedirs(crop_dir,exist_ok=True)
    
    for i, f in enumerate(img_files):
        try:
            
            img = Image.open(f)
            
            img  = img.convert("RGB") if channles==3 else img.convert("L")
            
            img_arr = np.array(img)
            
            for idx in range(n_crop):
                crop, valid = get_random_crop(img_arr)
                if valid :
                    crop_pil = Image.fromarray(crop).convert("RGB")
                    name = f"Img_{i:05d}_{idx:05d}.png"
                    im_path = os.path.join(crop_dir,name)
                    crop_pil.save(im_path)
        except Exception as e:
            print(f"Could not process {e}")        
    

['D:\\01-DATA\\dum\\c1\\000 - Copy (2) - Copy.png', 'D:\\01-DATA\\dum\\c1\\000 - Copy (2).png', 'D:\\01-DATA\\dum\\c1\\000 - Copy (3).png', 'D:\\01-DATA\\dum\\c1\\000 - Copy - Copy (2).png', 'D:\\01-DATA\\dum\\c1\\000 - Copy - Copy - Copy.png', 'D:\\01-DATA\\dum\\c1\\000 - Copy - Copy.png', 'D:\\01-DATA\\dum\\c1\\000 - Copy.png', 'D:\\01-DATA\\dum\\c1\\000.png']


## 4. Running the Pipeline

Set your image directory and run the cropping function. This will create a new 'Crop' folder with cropped patches from your images.

In [40]:
img_dir = r"D:\01-DATA\dum\c1"
# def loader(img_dir, crop_size = 200, channles = 3, n_crop = 5):

loader(img_dir, crop_size= 200, channles=3,n_crop=2)

['D:\\01-DATA\\dum\\c1\\000 - Copy (2) - Copy.png', 'D:\\01-DATA\\dum\\c1\\000 - Copy (2).png', 'D:\\01-DATA\\dum\\c1\\000 - Copy (3).png', 'D:\\01-DATA\\dum\\c1\\000 - Copy - Copy (2).png', 'D:\\01-DATA\\dum\\c1\\000 - Copy - Copy - Copy.png', 'D:\\01-DATA\\dum\\c1\\000 - Copy - Copy.png', 'D:\\01-DATA\\dum\\c1\\000 - Copy.png', 'D:\\01-DATA\\dum\\c1\\000.png']


## Interactive Cropping 

In [42]:
import os
import cv2
import numpy as np
from PIL import Image
from pathlib import Path

# Parameters
imagedir = r'D:\01-DATA\dum\c1'
img_format = 'png'  # or 'jpg', 'png'

save_dir = os.path.join(imagedir, "Cropped")
os.makedirs(save_dir, exist_ok=True)
images = list(Path(imagedir).glob(f'**/*.{img_format}'))

# Global variables for mouse callback
drawing = False
ix, iy = -1, -1
img_show, img_copy, img_crop = None, None, None

def draw(event, x, y, flags, param):
    global ix, iy, drawing, img_show, img_copy, img_crop
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        img_show = img_copy.copy()
        cv2.rectangle(img_show, (ix, iy), (x, y), (255, 255, 255), 2)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        x0, y0 = min(ix, x), min(iy, y)
        x1, y1 = max(ix, x), max(iy, y)
        img_show = img_copy.copy()
        cv2.rectangle(img_show, (x0, y0), (x1, y1), (255, 255, 255), 2)
        img_crop = img_copy[y0:y1, x0:x1]

print("Instructions:")
print("Draw rectangle with mouse, 's' to save, 'i' to skip, 'ESC' to quit.")

cv2.namedWindow("CropTool")
cv2.setMouseCallback("CropTool", draw)

for img_path in images:
    img = np.array(Image.open(img_path))
    if img.ndim == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    img_copy = img.copy()
    img_show = img.copy()
    img_crop = img.copy()

    while True:
        cv2.imshow("CropTool", img_show)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC
            cv2.destroyAllWindows()
            print('Done!')
            break
        elif key == ord('s') and img_crop is not None:
            out_path = os.path.join(save_dir, f"crop_{img_path.stem}.{img_format}")
            Image.fromarray(img_crop).convert("L").save(out_path)
            break
        elif key == ord('i'):
            break
    if key == 27:
        break

cv2.destroyAllWindows()


Instructions:
Draw rectangle with mouse, 's' to save, 'i' to skip, 'ESC' to quit.
Done!
